# part 1:Scraping data and build dataframes
  

### import and setup

In [1]:
# imports and setup 
from bs4 import BeautifulSoup
import requests

import time
import os
import random
import re

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.firefox.firefox_profile import FirefoxProfile


import pandas as pd
import scipy as sc
import numpy as np

import statsmodels.formula.api as sm

import matplotlib.pyplot as plt 
plt.style.use('ggplot')
%matplotlib inline  
plt.rcParams['figure.figsize'] = (10, 6) 

## first Data frame: yad2 scraping (apartments for sale)

In [ ]:
#scraping html of each yad2 page and add it to list
yad2_links=[]
yad2_tablesoup=[]


d = webdriver.Firefox(executable_path='PATH TO SELENUIM DRIVER HERE')

#save list of searchpage links
for i in range(1,700):  
    url = "https://URL HERE?page={}&searchbox=Course+Name&searchbox_zip=ZIP&distance=50&price_range=0&course_type=both&has_events=0".format(i)
    yad2_links.append(url) 
#scrape soup of pages  
count=0
for link in yad2_links:
    try:
        count=count+1
        if count<=7:
            d.get(link)
            time.sleep(random.uniform(1,3))
            elemnt=WebDriverWait(d,10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".feeditem.table"))) 
            expandable =d.find_elements_by_css_selector('.feeditem.table')  
            for item in expandable:
                item.click()
                      
            page_source=BeautifulSoup(d.page_source,'html.parser')
            yad2_tablesoup.append(page_source.find('div',{"class":"feed_list"}))
            print(len(yad2_tablesoup))
        else:
            count=0
            d.close()
            time.sleep(55)
            d = webdriver.Firefox(executable_path='PATH TO SELENUIM DRIVER HERE')
            pass
    except:
        print('next')
        pass

d.quit()


In [ ]:
#saving the scraped html's to text 
soup_prettified = [k.prettify() for k in yad2_tablesoup]#prettify the soupfied html elements in the list
list_with_word = [m+'BREAKHERE' for m in soup_prettified]# add the custom word to standout, I chose BREAKHERE
list_as_one = "".join(list_with_word)# save the giant string to .txt with# make one giant string, using the .join trick  
with open('saved_html.txt', 'w',encoding="utf-8") as file:
    file.write(list_as_one)

In [ ]:
# read the html's from the text file 
with open('fulllist.txt', 'r',encoding="utf-8") as f:
    content = f.read()
yad2_txt = content.split('BREAKHERE')# here's where to use the custom word 
yad2_full=[]

for i in yad2_txt:
    soup=BeautifulSoup(i,'html.parser')
    yad2_full.append(soup)
    #print(len(yad2_full))
    

In [3]:
# building objects for each post with BeautifulSoup
def build_post(post_soup,forsale_data,sold_data):
    post={
        'adress':'',
        'aprtment_type':'',
        'sub_area':'',
        'city':'',
        'rooms':'',
        'floor':'',
        'size':'',
        'price':'',
        #########
        'entry_date':'',
        'balcony':'',
        'building_floors':'',
        'state_of_asset':'',
        'parkings':'',
        'property_tax':'',
        #########
        'is_aircon':'',
        'is_borderwin':'',
        'is_elevator':'',
        'is_renovated':'',
        'is_storeroom':'',
        'is_furneter':'',
        ##########
        'date_update':'',
        'area_grade':'',
        'is_sold':'',
        }
    
    sold_post={
        'city':'',
        'sub_area':'',
        'date_sold':'',
        'adress':'',
        'rooms':'',
        'price_sold':'',
        'is_sold':''
        }
    
    #adress
    post['adress']=post_soup.find('span',{"class":"title"}).string.strip()
    
    #apt_type,sub_area,city
    temp0=post_soup.find('span',{"class":"subtitle"}).string.strip()
    temp1=temp0.split(', ')
    post['aprtment_type']=temp1[0]
    s='-'
    post['sub_area']=s.join(temp1[1:-1])
    post['city']=temp1[-1]
    
    #rooms,floor,size
    temp2=post_soup.find('div',{"class":"middle_col"})
    
    post['rooms']=temp2.find_all('span',{"class":"val"})[0].string.strip()
    post['floor']=temp2.find_all('span',{"class":"val"})[1].string.strip()
    post['size']=temp2.find_all('span',{"class":"val"})[2].string.strip()
    
    #price
    temp3=post_soup.find('div',{"class":"price"}).string.strip()
    post['price']=re.sub('[\,\ \₪]', '',temp3)
    
    #entry date,state of asset,balcony,parkings,building floors 
    temp4=post_soup.find('div',{"class":"info_items"})
    
    for dl in temp4.find_all("dl"):
        if (dl.find("dt").string.strip()=="תאריך כניסה"):
            post['entry_date']=dl.find("dd").string.strip()
        if (dl.find("dt").string.strip()=="מצב הנכס"):
            post['state_of_asset']=dl.find("dd").string.strip()
        if (dl.find("dt").string.strip()=="מרפסות"):
            post['balcony']=dl.find("dd").string.strip()
        if (dl.find("dt").string.strip()=="חניות"):
            post['parkings']=dl.find("dd").string.strip()
        if (dl.find("dt").string.strip()=="קומות בבנין"):
            post['building_floors']=dl.find("dd").string.strip()
        if (dl.find("dt").string.strip()=="ארנונה לחודשיים"):
            post['property_tax']=dl.find("dd").string.strip()
     
    temp5=post_soup.find_all('div',{"class":"info_feature delete"})
    post['is_renovated']=1
    post['is_aircon']=1
    post['is_elevator']=1
    post['is_furneter']=1
    post['is_borderwin']=1
    post['is_storeroom']=1
    
    for item in temp5:
        if (item.find("span").string.strip()=="משופצת"):
            post['is_renovated']=0
        if (item.find("span").string.strip()=="מיזוג"):
            post['is_aircon']=0
        if (item.find("span").string.strip()=="מעלית"):
            post['is_elevator']=0
        if (item.find("span").string.strip()=="ריהוט"):
            post['is_furneter']=0
        if (item.find("span").string.strip()=="סורגים"):
            post['is_borderwin']=0
        if (item.find("span").string.strip()=="מחסן"):
            post['is_storeroom']=0
 
    #date ipdate
    post['date_update']=post_soup.find('div',{"class":"showDateInLobby"}).string.strip()
    
    post['is_sold']=0
    
    temp6=post_soup.find('div',{"class":"table_container"})
    if temp6 is not None:
        for row in temp6.find_all('div',{"class":"details_content"}):
            sold_post={}
            sold_post['city']=post['city']
            sold_post['sub_area']=post['sub_area']
            sold_post['is_sold']=1
            
            temp7=row.find_all('span')[0].string.strip()
            sold_post['date_sold']=re.sub('[.]', '/',temp7)
            sold_post['adress']=row.find_all('span')[1].string.strip()
            sold_post['rooms']=row.find_all('span')[2].string.strip()
            temp8=row.find_all('span')[3].string.strip()
            sold_post['price_sold']=re.sub('[,]', '',temp8)
            sold_data.append(sold_post)
            
    
    forsale_data.append(post)

In [ ]:
#running the func on the searchpages soup list
forsale_data=[]
sold_data=[]
for i in yad2_full:
    for post in i.find_all('div',{"class":"feeditem table"}):
        if len(post.text)>900:
            build_post(post,forsale_data,sold_data)
            #print(len(forsale_data),"::",len(sold_data))
            
  

In [8]:
#converting the objects creted to dataframe and saving as csv 
forsale_yad2_df=pd.DataFrame(forsale_data)
forsale_yad2_df.to_csv('NAME HERE')

sold_yad2_df=pd.DataFrame(sold_data)
sold_yad2_df.to_csv('NAME HERE')

In [9]:
sold_yad2_df

,city,sub_area,is_sold,date_sold,adress,rooms,price_sold
0,ראשון לציון,ראשונים,1,26/02/20,אהרון קרון 26,4,1840000
1,ראשון לציון,ראשונים,1,10/02/20,הרצל 107,4,1600000
2,ראשון לציון,ראשונים,1,28/01/20,אהרון קרון 19,4,2565872
3,קרית גת,כרמי גת,1,03/03/20,הנרייטה סאלד 67,4,1230000
4,קרית גת,כרמי גת,1,24/02/20,הנרייטה סאלד 18,4,1245000
...,...,...,...,...,...,...,...
42334,חדרה,ויצמן,1,16/12/19,"קדימה 2, גבעת אולגה",7,2050000
42335,חדרה,ויצמן,1,11/12/19,"נחל פולג 14, בית אליעזר",7,2650000
42336,נהריה,עין שרה,1,15/03/20,השקד 25,5,1090000
42337,נהריה,עין שרה,1,05/03/20,לוטוס 104,5,2300000


In [10]:
forsale_yad2_df

,adress,aprtment_type,sub_area,city,rooms,floor,size,price,entry_date,balcony,...,property_tax,is_aircon,is_borderwin,is_elevator,is_renovated,is_storeroom,is_furneter,date_update,area_grade,is_sold
0,טיומקין 1,דירה,ראשונים,ראשון לציון,4.5,6,110,2200000,כניסה מיידית,1,...,,1,0,1,0,0,0,10/06/2020,,0
1,אחלמה 11,דירה,כרמי גת,קרית גת,4,3,85,1190000,01/12/2021,2,...,,0,0,1,0,0,0,10/06/2020,,0
2,סנפיר 3,דירה,נווה חן,תל אביב יפו,2,8,46,1690000,לא צוין תאריך כניסה,,...,,1,0,1,0,0,1,10/06/2020,,0
3,רב אלוף דן שומרון 25,דירה,המשולש הגדול,רמת גן,4,4,108,3100000,כניסה מיידית,1,...,,1,0,1,0,1,0,10/06/2020,,0
4,נחל הצבי 77,דירה,קרית רבין,מגדל העמק,4,1,119,1050000,לא צוין תאריך כניסה,1,...,,1,0,1,0,1,0,10/06/2020,,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28772,לא צוין רחוב,דו משפחתי,,מזכרת בתיה,6,קרקע,250,3600000,לא צוין תאריך כניסה,1,...,,0,0,0,0,0,0,16/06/2020,,0
28773,נוף העמק,פרטי/קוטג',נוף העמק,מגדל העמק,6,קרקע,500,2450000,כניסה מיידית,1,...,,1,0,0,1,0,0,16/06/2020,,0
28774,"ריב""ל",דירה,נאות שקמה,ראשון לציון,4,7,105,לאצויןמחיר,לא צוין תאריך כניסה,1,...,,1,1,0,1,0,0,16/06/2020,,0
28775,הרופאים,פרטי/קוטג',יבנאל,יבנאל,5,קרקע,600,1390000,כניסה מיידית,1,...,,1,0,0,0,1,0,16/06/2020,,0
